In [21]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import joblib
import copy

# -------------------------------
# 1. 데이터 로드 및 전처리
# -------------------------------
data = pd.read_csv('train2.csv', encoding='cp949')
X = data.drop(columns=['ID', '시술 시기 코드', '시술 유형', '임신 성공 여부'])
y = data['임신 성공 여부']

# -------------------------------
# 2. Ensemble 학습을 위한 설정
# -------------------------------
n_iterations = 500   # 전체 데이터를 500번 사용
max_depth_start = 1
max_depth_end = 1000
patience = 1         # 연속 개선이 없으면 early stopping (여기서는 매우 짧게 설정)

ensemble_models = []        # 각 반복의 최적 모델 저장 리스트
ensemble_best_depths = []   # 각 모델의 최적 max_depth 저장
ensemble_train_auc = []     # 각 모델의 학습 ROC AUC 저장
ensemble_val_auc = []       # 각 모델의 검증 ROC AUC 저장

# -------------------------------
# 3. 여러 번의 무작위 데이터 분할 및 모델 학습
# -------------------------------
for i in range(n_iterations):
    print(f"\n=== Ensemble Model {i+1}/{n_iterations} ===")
    # 무작위 분할 (매번 다른 분할)
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.2, stratify=y
    )
    
    # 파이프라인 구성: StandardScaler와 DecisionTreeClassifier
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('dt', DecisionTreeClassifier())
    ])
    
    patience_counter = 0   # early stopping 카운터 초기화
    best_score = -np.inf     # 검증 ROC AUC 최적값 초기화
    best_depth = None
    best_model = None
    
    # max_depth를 1부터 max_depth_end까지 증가시키며 모델 학습 및 평가
    for depth in range(max_depth_start, max_depth_end + 1):
        pipeline.set_params(dt__max_depth=depth)
        pipeline.fit(X_train, y_train)
        
        # 학습 및 검증 데이터 ROC AUC 계산
        y_train_prob = pipeline.predict_proba(X_train)[:, 1]
        train_score = roc_auc_score(y_train, y_train_prob)
        y_val_prob = pipeline.predict_proba(X_val)[:, 1]
        current_score = roc_auc_score(y_val, y_val_prob)
        
        # 각 depth 반복 시점의 결과 출력 (트래킹)
        print(f"Depth: {depth}, Train ROC AUC: {train_score:.4f}, Valid ROC AUC: {current_score:.4f}")
        
        # 검증 ROC AUC가 개선되면 갱신
        if current_score > best_score:
            best_score = current_score
            best_depth = depth
            best_model = copy.deepcopy(pipeline)
            patience_counter = 0  # 개선 시 카운터 리셋
        else:
            patience_counter += 1
        
        # 연속 개선이 없으면 조기 종료
        if patience_counter >= patience:
            print(f"Early stopping triggered at depth: {depth}")
            break

    # 해당 반복의 최종 학습 데이터 ROC AUC 재계산 (선택 사항)
    y_train_prob = best_model.predict_proba(X_train)[:, 1]
    final_train_score = roc_auc_score(y_train, y_train_prob)
    
    print(f"최적 Depth: {best_depth}, 최종 Train ROC AUC: {final_train_score:.4f}, 최종 Valid ROC AUC: {best_score:.4f}")
    
    ensemble_models.append(best_model)
    ensemble_best_depths.append(best_depth)
    ensemble_train_auc.append(final_train_score)
    ensemble_val_auc.append(best_score)

# -------------------------------
# 4. Ensemble 평가 (평균 성능 확인)
# -------------------------------
avg_train_auc = np.mean(ensemble_train_auc)
avg_val_auc = np.mean(ensemble_val_auc)
print("\nEnsemble 평균 성능:")
print(f"평균 Train ROC AUC: {avg_train_auc:.4f}")
print(f"평균 Valid ROC AUC: {avg_val_auc:.4f}")

# 전체 모델 저장 (나중에 재사용 가능)
joblib.dump(ensemble_models, "ensemble_deep_decision_trees.pkl")
print("\nEnsemble 모델이 'ensemble_deep_decision_trees.pkl'로 저장되었습니다.")

# -------------------------------
# 5. 상위 및 전체 모델 기반으로 테스트 데이터 예측
# -------------------------------
# validation ROC AUC를 기준으로 내림차순 정렬할 인덱스를 구합니다.
val_auc_array = np.array(ensemble_val_auc)
sorted_indices = np.argsort(val_auc_array)[::-1]  # 내림차순 (높은 성능순)

# 사용할 모델 개수별 인덱스 추출
top100_indices = sorted_indices[:100]
top200_indices = sorted_indices[:200]
top300_indices = sorted_indices[:300]
all_indices = np.arange(len(ensemble_models))  # 전체 500개 모델

# 테스트 데이터 로드 및 전처리
test_data = pd.read_csv('test2.csv', encoding='cp949')
X_test = test_data.drop(columns=['ID', '시술 시기 코드', '시술 유형', '임신 성공 여부'], errors='ignore')

def predict_with_ensemble(model_indices, models, X_test, threshold=0.5):
    probs_list = []
    for idx in model_indices:
        probs = models[idx].predict_proba(X_test)[:, 1]
        probs_list.append(probs)
    avg_probs = np.mean(probs_list, axis=0)
    preds = (avg_probs >= threshold).astype(int)
    return preds

# 각 앙상블 결과로 예측
top100_preds = predict_with_ensemble(top100_indices, ensemble_models, X_test)
top200_preds = predict_with_ensemble(top200_indices, ensemble_models, X_test)
top300_preds = predict_with_ensemble(top300_indices, ensemble_models, X_test)
all_preds    = predict_with_ensemble(all_indices,    ensemble_models, X_test)

# 예측 결과를 test_data에 추가
test_data['임신 성공 여부 예측_top100'] = top100_preds
test_data['임신 성공 여부 예측_top200'] = top200_preds
test_data['임신 성공 여부 예측_top300'] = top300_preds
test_data['임신 성공 여부 예측_all']     = all_preds

# 각각의 파일로 저장 (원하는 대로 컬럼 선택 가능)
test_data[['ID', '임신 성공 여부 예측_top100']].to_csv('ensemble_test_predictions_top100.csv', index=False, encoding='cp949')
test_data[['ID', '임신 성공 여부 예측_top200']].to_csv('ensemble_test_predictions_top200.csv', index=False, encoding='cp949')
test_data[['ID', '임신 성공 여부 예측_top300']].to_csv('ensemble_test_predictions_top300.csv', index=False, encoding='cp949')
test_data[['ID', '임신 성공 여부 예측_all']].to_csv('ensemble_test_predictions_all.csv', index=False, encoding='cp949')

print("테스트 데이터 예측 결과가 각각 'ensemble_test_predictions_top100.csv',")
print("'ensemble_test_predictions_top200.csv', 'ensemble_test_predictions_top300.csv',")
print("그리고 'ensemble_test_predictions_all.csv' 파일에 저장되었습니다.")


=== Ensemble Model 1/500 ===
Depth: 1, Train ROC AUC: 0.5978, Valid ROC AUC: 0.5978
Depth: 2, Train ROC AUC: 0.6635, Valid ROC AUC: 0.6648
Depth: 3, Train ROC AUC: 0.6909, Valid ROC AUC: 0.6920
Depth: 4, Train ROC AUC: 0.7022, Valid ROC AUC: 0.7040
Depth: 5, Train ROC AUC: 0.7103, Valid ROC AUC: 0.7117
Depth: 6, Train ROC AUC: 0.7152, Valid ROC AUC: 0.7157
Depth: 7, Train ROC AUC: 0.7187, Valid ROC AUC: 0.7183
Depth: 8, Train ROC AUC: 0.7213, Valid ROC AUC: 0.7177
Early stopping triggered at depth: 8
최적 Depth: 7, 최종 Train ROC AUC: 0.7187, 최종 Valid ROC AUC: 0.7183

=== Ensemble Model 2/500 ===
Depth: 1, Train ROC AUC: 0.5975, Valid ROC AUC: 0.5990
Depth: 2, Train ROC AUC: 0.6632, Valid ROC AUC: 0.6660
Depth: 3, Train ROC AUC: 0.6908, Valid ROC AUC: 0.6924
Depth: 4, Train ROC AUC: 0.7023, Valid ROC AUC: 0.7038
Depth: 5, Train ROC AUC: 0.7104, Valid ROC AUC: 0.7117
Depth: 6, Train ROC AUC: 0.7152, Valid ROC AUC: 0.7161
Depth: 7, Train ROC AUC: 0.7189, Valid ROC AUC: 0.7177
Depth: 8, Trai

In [22]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import joblib
import copy

# -------------------------------
# 1. 데이터 로드 및 전처리
# -------------------------------
data = pd.read_csv('train2.csv', encoding='cp949')
X = data.drop(columns=['ID', '시술 시기 코드', '시술 유형', '임신 성공 여부',
                '여성 주 불임 원인', '부부 주 불임 원인', '부부 부 불임 원인',
                'IVF 시술 횟수', 'IVF 출산 횟수', '혼합된 난자 수',
                '동결 배아 사용 여부', '신선 배아 사용 여부'])
y = data['임신 성공 여부']

# -------------------------------
# 2. Ensemble 학습을 위한 설정
# -------------------------------
n_iterations = 500   # 전체 데이터를 500번 사용
max_depth_start = 1
max_depth_end = 1000
patience = 1         # 연속 개선이 없으면 early stopping (여기서는 매우 짧게 설정)

ensemble_models = []        # 각 반복의 최적 모델 저장 리스트
ensemble_best_depths = []   # 각 모델의 최적 max_depth 저장
ensemble_train_auc = []     # 각 모델의 학습 ROC AUC 저장
ensemble_val_auc = []       # 각 모델의 검증 ROC AUC 저장

# -------------------------------
# 3. 여러 번의 무작위 데이터 분할 및 모델 학습
# -------------------------------
for i in range(n_iterations):
    print(f"\n=== Ensemble Model {i+1}/{n_iterations} ===")
    # 무작위 분할 (매번 다른 분할)
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.2, stratify=y
    )
    
    # 파이프라인 구성: StandardScaler와 DecisionTreeClassifier
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('dt', DecisionTreeClassifier())
    ])
    
    patience_counter = 0   # early stopping 카운터 초기화
    best_score = -np.inf     # 검증 ROC AUC 최적값 초기화
    best_depth = None
    best_model = None
    
    # max_depth를 1부터 max_depth_end까지 증가시키며 모델 학습 및 평가
    for depth in range(max_depth_start, max_depth_end + 1):
        pipeline.set_params(dt__max_depth=depth)
        pipeline.fit(X_train, y_train)
        
        # 학습 및 검증 데이터 ROC AUC 계산
        y_train_prob = pipeline.predict_proba(X_train)[:, 1]
        train_score = roc_auc_score(y_train, y_train_prob)
        y_val_prob = pipeline.predict_proba(X_val)[:, 1]
        current_score = roc_auc_score(y_val, y_val_prob)
        
        # 각 depth 반복 시점의 결과 출력 (트래킹)
        print(f"Depth: {depth}, Train ROC AUC: {train_score:.4f}, Valid ROC AUC: {current_score:.4f}")
        
        # 검증 ROC AUC가 개선되면 갱신
        if current_score > best_score:
            best_score = current_score
            best_depth = depth
            best_model = copy.deepcopy(pipeline)
            patience_counter = 0  # 개선 시 카운터 리셋
        else:
            patience_counter += 1
        
        # 연속 개선이 없으면 조기 종료
        if patience_counter >= patience:
            print(f"Early stopping triggered at depth: {depth}")
            break

    # 해당 반복의 최종 학습 데이터 ROC AUC 재계산 (선택 사항)
    y_train_prob = best_model.predict_proba(X_train)[:, 1]
    final_train_score = roc_auc_score(y_train, y_train_prob)
    
    print(f"최적 Depth: {best_depth}, 최종 Train ROC AUC: {final_train_score:.4f}, 최종 Valid ROC AUC: {best_score:.4f}")
    
    ensemble_models.append(best_model)
    ensemble_best_depths.append(best_depth)
    ensemble_train_auc.append(final_train_score)
    ensemble_val_auc.append(best_score)

# -------------------------------
# 4. Ensemble 평가 (평균 성능 확인)
# -------------------------------
avg_train_auc = np.mean(ensemble_train_auc)
avg_val_auc = np.mean(ensemble_val_auc)
print("\nEnsemble 평균 성능:")
print(f"평균 Train ROC AUC: {avg_train_auc:.4f}")
print(f"평균 Valid ROC AUC: {avg_val_auc:.4f}")

# 전체 모델 저장 (나중에 재사용 가능)
joblib.dump(ensemble_models, "ensemble_deep_decision_trees.pkl")
print("\nEnsemble 모델이 'ensemble_deep_decision_trees.pkl'로 저장되었습니다.")

# -------------------------------
# 5. 상위 및 전체 모델 기반으로 테스트 데이터 예측
# -------------------------------
# validation ROC AUC를 기준으로 내림차순 정렬할 인덱스를 구합니다.
val_auc_array = np.array(ensemble_val_auc)
sorted_indices = np.argsort(val_auc_array)[::-1]  # 내림차순 (높은 성능순)

# 사용할 모델 개수별 인덱스 추출
top100_indices = sorted_indices[:100]
top200_indices = sorted_indices[:200]
top300_indices = sorted_indices[:300]
all_indices = np.arange(len(ensemble_models))  # 전체 500개 모델

# 테스트 데이터 로드 및 전처리
test_data = pd.read_csv('test2.csv', encoding='cp949')
X_test = test_data.drop(columns=['ID', '시술 시기 코드', '시술 유형', '임신 성공 여부',
                '여성 주 불임 원인', '부부 주 불임 원인', '부부 부 불임 원인',
                'IVF 시술 횟수', 'IVF 출산 횟수', '혼합된 난자 수',
                '동결 배아 사용 여부', '신선 배아 사용 여부'], errors='ignore')

def predict_with_ensemble(model_indices, models, X_test, threshold=0.5):
    probs_list = []
    for idx in model_indices:
        probs = models[idx].predict_proba(X_test)[:, 1]
        probs_list.append(probs)
    avg_probs = np.mean(probs_list, axis=0)
    preds = (avg_probs >= threshold).astype(int)
    return preds

# 각 앙상블 결과로 예측
top100_preds = predict_with_ensemble(top100_indices, ensemble_models, X_test)
top200_preds = predict_with_ensemble(top200_indices, ensemble_models, X_test)
top300_preds = predict_with_ensemble(top300_indices, ensemble_models, X_test)
all_preds    = predict_with_ensemble(all_indices,    ensemble_models, X_test)

# 예측 결과를 test_data에 추가
test_data['임신 성공 여부 예측_top100'] = top100_preds
test_data['임신 성공 여부 예측_top200'] = top200_preds
test_data['임신 성공 여부 예측_top300'] = top300_preds
test_data['임신 성공 여부 예측_all']     = all_preds

# 각각의 파일로 저장 (원하는 대로 컬럼 선택 가능)
test_data[['ID', '임신 성공 여부 예측_top100']].to_csv('ensemble_test_predictions_top100_feature_selection.csv', index=False, encoding='cp949')
test_data[['ID', '임신 성공 여부 예측_top200']].to_csv('ensemble_test_predictions_top200_feature_selection.csv', index=False, encoding='cp949')
test_data[['ID', '임신 성공 여부 예측_top300']].to_csv('ensemble_test_predictions_top300_feature_selection.csv', index=False, encoding='cp949')
test_data[['ID', '임신 성공 여부 예측_all']].to_csv('ensemble_test_predictions_all_feature_selection.csv', index=False, encoding='cp949')

print("테스트 데이터 예측 결과가 각각 'ensemble_test_predictions_top100.csv',")
print("'ensemble_test_predictions_top200.csv', 'ensemble_test_predictions_top300.csv',")
print("그리고 'ensemble_test_predictions_all.csv' 파일에 저장되었습니다.")


=== Ensemble Model 1/500 ===
Depth: 1, Train ROC AUC: 0.5977, Valid ROC AUC: 0.5981
Depth: 2, Train ROC AUC: 0.6651, Valid ROC AUC: 0.6637
Depth: 3, Train ROC AUC: 0.6914, Valid ROC AUC: 0.6884
Depth: 4, Train ROC AUC: 0.7042, Valid ROC AUC: 0.7001
Depth: 5, Train ROC AUC: 0.7113, Valid ROC AUC: 0.7073
Depth: 6, Train ROC AUC: 0.7161, Valid ROC AUC: 0.7125
Depth: 7, Train ROC AUC: 0.7188, Valid ROC AUC: 0.7155
Depth: 8, Train ROC AUC: 0.7216, Valid ROC AUC: 0.7167
Depth: 9, Train ROC AUC: 0.7247, Valid ROC AUC: 0.7152
Early stopping triggered at depth: 9
최적 Depth: 8, 최종 Train ROC AUC: 0.7216, 최종 Valid ROC AUC: 0.7167

=== Ensemble Model 2/500 ===
Depth: 1, Train ROC AUC: 0.5974, Valid ROC AUC: 0.5992
Depth: 2, Train ROC AUC: 0.6630, Valid ROC AUC: 0.6668
Depth: 3, Train ROC AUC: 0.6907, Valid ROC AUC: 0.6930
Depth: 4, Train ROC AUC: 0.7022, Valid ROC AUC: 0.7040
Depth: 5, Train ROC AUC: 0.7101, Valid ROC AUC: 0.7131
Depth: 6, Train ROC AUC: 0.7150, Valid ROC AUC: 0.7165
Depth: 7, Trai

In [ ]:
'ID', '시술 시기 코드', '시술 유형', '임신 성공 여부',
                '여성 주 불임 원인', '부부 주 불임 원인', '부부 부 불임 원인',
                'IVF 시술 횟수', 'IVF 출산 횟수', '혼합된 난자 수',
                '동결 배아 사용 여부', '신선 배아 사용 여부'

In [ ]:
0.72 이상인 애들만 진행하기